# Stock Market Prediction Model of the Magnificient Seven 

#pip install the dependencies from requirements. May take up to 12 minutes

In [27]:
!python3 -m pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


### importing our libraries after pipinstall

In [28]:

#python data manipulation
import pandas as pd
import numpy as np

#data visualizaiton tools, EDA
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

#mysql and Cassandra
import mysql.connector                         
from cassandra.cluster import Cluster          

#ARIMA
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

#LSTM Neural network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf

# our finance data
import yfinance as yf



## Extracting the Historical Data.


 ### The historical aspect of the data is starting from January 1st 2020 to yesterday (april 6th), We can adjust this to most recent always. We extracted the 7 companies and start with first 7 features from our project proposal: Date/time, open price, low price, high price, adjusted close price, trading volume

In [29]:
def extract_data(tickers, start_date="2020-01-01", end_date="2025-04-06", interval="1d"):
    # importing the yfinance api data, making sure the autoadjust is off because it overrides the close value if its on
    df = yf.download(tickers, start=start_date, end=end_date, interval=interval, auto_adjust=False)
    # tidying the data to make it better for analysis and transformation later
    df.columns = df.columns.swaplevel(0, 1)
    df = df.sort_index(axis=1, level=0)
    df_flat = df.stack(level=0, future_stack=True).reset_index()  
    df_flat.rename(columns={'level_0': 'Date'}, inplace=True)
    # renaming 'Adj Close' to 'Adj_Close' for consistency
    if 'Adj Close' in df_flat.columns:
        df_flat.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)
    # adding the previous day's close price per ticker
    df_flat['Previous_Close'] = df_flat.groupby('Ticker')['Close'].shift(1)
    # selecting the columns we expect and need
    expected_cols = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Previous_Close', 'Volume']
    existing_cols = [col for col in expected_cols if col in df_flat.columns]
    return df_flat[existing_cols]


In [30]:
#giving yfinance a list of companies to return a list of data of. AND displaying the df
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA"]
df_raw = extract_data(tickers)
display(df_raw)

[*********************100%***********************]  7 of 7 completed


Price,Date,Ticker,Open,High,Low,Close,Adj_Close,Previous_Close,Volume
0,2020-01-02,AAPL,74.059998,75.150002,73.797501,75.087502,72.716080,NaN,135480400
1,2020-01-02,AMZN,93.750000,94.900497,93.207497,94.900497,94.900497,NaN,80580000
2,2020-01-02,GOOGL,67.420502,68.433998,67.324501,68.433998,68.108376,NaN,27278000
3,2020-01-02,META,206.750000,209.789993,206.270004,209.779999,208.795929,NaN,12077100
4,2020-01-02,MSFT,158.779999,160.729996,158.330002,160.619995,153.323257,NaN,22622100
...,...,...,...,...,...,...,...,...,...
9249,2025-04-04,GOOGL,148.009995,151.070007,145.380005,145.600006,145.600006,150.720001,62259500
9250,2025-04-04,META,506.619995,518.000000,494.200012,504.730011,504.730011,531.619995,38589800
9251,2025-04-04,MSFT,364.130005,374.589996,359.480011,359.839996,359.839996,373.109985,49209900
9252,2025-04-04,NVDA,98.910004,100.129997,92.110001,94.309998,94.309998,101.800003,532273800


## Historical Data: Data Cleaning: 
### We will clean the data by taking care of any null values. We can use back and forward filling if there is a value missing from a float/int value. Otherwise we will drop the value if its in the date or Ticker/ or use the average of the past few days for the column. Additionally if there are are any duplicate records for a company and a speciifc date, one should be kept while rest dropped.

In [ ]:
nans = df_raw.isna().sum()
print(nans)
def cleaning(df):
    # Step 1: Drop duplicates (keep the first entry for each Ticker-Date pair)
    df = df.drop_duplicates(subset=['Date', 'Ticker'], keep='first')

    # Step 2: Define numeric columns to clean (excluding 'Date' and 'Ticker')
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume', 'Previous_Close']
    
    # Step 3: Fill missing values for numeric columns
    for col in numeric_cols:
        # Forward fill and backward fill first
        df[col] = df.groupby('Ticker')[col].transform(lambda x: x.ffill().bfill())
        
        # Fill remaining NaNs with rolling mean (3-day window)
        df[col] = df.groupby('Ticker')[col].transform(
            lambda x: x.fillna(x.rolling(window=3, min_periods=1).mean())
        )

    # Step 4: Ensure 'Previous_Close' is properly filled (first entry has no previous data)
    df['Previous_Close'] = df.groupby('Ticker')['Previous_Close'].ffill()

    # Step 5: Drop rows with nulls in 'Date' or 'Ticker' (shouldn't exist, but safety check)
    df = df.dropna(subset=['Date', 'Ticker'])

    return df
# Apply cleaning
df_cleaned = cleaning(df_raw)

# Verify results
print("Null values after cleaning:")
print(df_cleaned.isna().sum())
display(df_cleaned)

Price
Date              0
Ticker            0
Open              0
High              0
Low               0
Close             0
Adj_Close         0
Previous_Close    7
Volume            0
dtype: int64
Null values after cleaning:
Price
Date              0
Ticker            0
Open              0
High              0
Low               0
Close             0
Adj_Close         0
Previous_Close    0
Volume            0
dtype: int64


Price,Date,Ticker,Open,High,Low,Close,Adj_Close,Previous_Close,Volume
0,2020-01-02,AAPL,74.059998,75.150002,73.797501,75.087502,72.716080,75.087502,135480400
1,2020-01-02,AMZN,93.750000,94.900497,93.207497,94.900497,94.900497,94.900497,80580000
2,2020-01-02,GOOGL,67.420502,68.433998,67.324501,68.433998,68.108376,68.433998,27278000
3,2020-01-02,META,206.750000,209.789993,206.270004,209.779999,208.795929,209.779999,12077100
4,2020-01-02,MSFT,158.779999,160.729996,158.330002,160.619995,153.323257,160.619995,22622100
...,...,...,...,...,...,...,...,...,...
9249,2025-04-04,GOOGL,148.009995,151.070007,145.380005,145.600006,145.600006,150.720001,62259500
9250,2025-04-04,META,506.619995,518.000000,494.200012,504.730011,504.730011,531.619995,38589800
9251,2025-04-04,MSFT,364.130005,374.589996,359.480011,359.839996,359.839996,373.109985,49209900
9252,2025-04-04,NVDA,98.910004,100.129997,92.110001,94.309998,94.309998,101.800003,532273800


## Historical Data, Feature Engineering:

### We are going to be transforming the tidy dataframe by doing some feature engineering. We will create the following fields into our table by manipulating the prexisting data from df_cleaned: 
1. Simple Moving Average (SMA)
2. Exponential Moving Average (EMA)
3. Relative Strength Index (RSI)
4. Bollinger Bands
5. MACD (Moving Average Convergence Divergence)
6. On-Balance Volume (OBV)
7. Volatility (ATR - Average True Range)


In [ ]:
def transform(df):
    # Ensure data is sorted by Ticker and Date
    df = df.sort_values(['Ticker', 'Date']).reset_index(drop=True)
    
    # 1. Simple Moving Average (SMA)
    #uses 20 periods of data (NaN till 19 row)
    df['SMA_20'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).mean())
    
    # 2. Exponential Moving Average (EMA)
    df['EMA_20'] = df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=20, adjust=False).mean())
    
    # 3. Relative Strength Index (RSI)
    #uses 14 periods of data (NaN till 13 row)
    def compute_rsi(data, period=14):
        delta = data.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=period).mean()
        avg_loss = loss.rolling(window=period).mean()
        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))
    df['RSI_14'] = df.groupby('Ticker')['Close'].transform(compute_rsi)
    
    # 4. Bollinger Bands
    #NaN till we get SMA values (19th row)
    df['BB_Middle'] = df['SMA_20']
    df['BB_Std'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).std())
    df['BB_Upper'] = df['BB_Middle'] + 2 * df['BB_Std']
    df['BB_Lower'] = df['BB_Middle'] - 2 * df['BB_Std']
    
    # 5. MACD
    def compute_macd(data):
        ema12 = data.ewm(span=12, adjust=False).mean()
        ema26 = data.ewm(span=26, adjust=False).mean()
        return ema12 - ema26
    df['MACD'] = df.groupby('Ticker')['Close'].transform(compute_macd)
    df['MACD_Signal'] = df.groupby('Ticker')['MACD'].transform(lambda x: x.ewm(span=9, adjust=False).mean())
    
    # # 6. On-Balance Volume (OBV)
    # def compute_obv(group):
    #     obv = (np.where(group['Close'].diff() > 0, group['Volume'],
    #             np.where(group['Close'].diff() < 0, -group['Volume'], 0))).cumsum()
    #     return obv
    
    # df['OBV'] = df.groupby('Ticker', group_keys=False).apply(compute_obv).reset_index(drop=True)
    
    # # 7. Average True Range (ATR)
    # def compute_atr(group):
    #     high_low = group['High'] - group['Low']
    #     high_close = (group['High'] - group['Close'].shift()).abs()
    #     low_close = (group['Low'] - group['Close'].shift()).abs()
    #     tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    #     return tr.rolling(window=14).mean()
    
    # df['ATR_14'] = df.groupby('Ticker', group_keys=False).apply(compute_atr).reset_index(drop=True)
    
    # # Forward fill missing values created by rolling windows
    # df = df.groupby('Ticker').ffill().reset_index(drop=True)
    
    return df

# Apply transformation to cleaned data
df_transformed = transform(df_cleaned)
display(df_transformed[['Date', 'Ticker', 'SMA_20', 'EMA_20', 'RSI_14', 'BB_Upper', 'BB_Lower', 'MACD']].head(n=20)) #only shows APPL b/c all other tickers appear later. 
#add in 'OBV' and 'ATR_14' when fixed 

#to filter a specific ticker: 
display(df_transformed[df_transformed['Ticker'] == 'MSFT'].iloc[20:25])

#to shuffle/show random rows: 
display(df_transformed.sample(10))  # Show random rows

# count = len(df_transformed)
# print(count)


Price,Date,Ticker,SMA_20,EMA_20,RSI_14,BB_Upper,BB_Lower,MACD
0,2020-01-02,AAPL,NaN,75.087502,NaN,NaN,NaN,0.000000
1,2020-01-03,AAPL,NaN,75.017977,NaN,NaN,NaN,-0.058234
2,2020-01-06,AAPL,NaN,75.011503,NaN,NaN,NaN,-0.055930
3,2020-01-07,AAPL,NaN,74.972075,NaN,NaN,NaN,-0.081607
4,2020-01-08,AAPL,NaN,75.050687,NaN,NaN,NaN,-0.005068
5,2020-01-09,AAPL,NaN,75.275145,NaN,NaN,NaN,0.183389
6,2020-01-10,AAPL,NaN,75.494893,NaN,NaN,NaN,0.342911
7,2020-01-13,AAPL,NaN,75.851570,NaN,NaN,NaN,0.596207
8,2020-01-14,AAPL,NaN,76.072372,NaN,NaN,NaN,0.702507
9,2020-01-15,AAPL,NaN,76.240241,NaN,NaN,NaN,0.751062


Price,Date,Ticker,Open,High,Low,Close,Adj_Close,Previous_Close,Volume,SMA_20,EMA_20,RSI_14,BB_Middle,BB_Std,BB_Upper,BB_Lower,MACD,MACD_Signal
5308,2020-01-31,MSFT,172.210007,172.399994,169.580002,170.229996,162.496689,172.779999,36142700,164.167999,165.244291,65.892017,164.167999,3.935323,172.038644,156.297353,2.175122,1.508417
5309,2020-02-03,MSFT,170.429993,174.500000,170.399994,174.380005,166.458176,170.229996,30107000,164.955999,166.114359,68.389658,164.955999,4.324559,173.605117,156.306881,2.627071,1.732148
5310,2020-02-04,MSFT,177.139999,180.639999,176.309998,180.119995,171.937408,174.380005,36433300,166.010499,167.448229,75.869974,166.010499,5.271194,176.552887,155.468110,3.409114,2.067541
5311,2020-02-05,MSFT,184.029999,184.199997,178.410004,179.899994,171.727386,180.119995,39186300,167.126498,168.634112,74.631681,167.126498,5.734751,178.596000,155.656997,3.965426,2.447118
5312,2020-02-06,MSFT,180.970001,183.820007,180.059998,183.630005,175.287964,179.899994,27751400,168.303499,170.062292,75.172992,168.303499,6.569494,181.442486,155.164511,4.653644,2.888423


Price,Date,Ticker,Open,High,Low,Close,Adj_Close,Previous_Close,Volume,SMA_20,EMA_20,RSI_14,BB_Middle,BB_Std,BB_Upper,BB_Lower,MACD,MACD_Signal
7346,2022-12-02,NVDA,16.659000,16.933001,16.445000,16.875999,16.862185,17.135000,371389000,15.770650,15.690009,54.911432,15.770650,0.950766,17.672183,13.869117,0.795527,0.750189
2209,2023-07-13,AMZN,134.039993,134.669998,132.710007,134.300003,134.300003,130.800003,61170900,128.633500,128.216767,59.444724,128.633500,2.226864,133.087227,124.179772,2.690890,2.873994
5340,2020-03-18,MSFT,138.000000,146.000000,135.020004,140.399994,134.387909,146.570007,81593200,160.673499,157.805539,43.718205,160.673499,13.183556,187.040612,134.306387,-7.963194,-5.458603
5404,2020-06-18,MSFT,194.000000,196.490005,194.000000,196.320007,188.436676,194.240005,23061600,187.013501,188.025408,66.366153,187.013501,4.929516,196.872533,177.154469,3.854948,3.352926
6473,2024-09-18,MSFT,435.000000,436.029999,430.410004,430.809998,429.089020,435.149994,18898000,417.547502,420.757042,66.906471,417.547502,9.565503,436.678507,398.416497,2.892459,-0.197340
5787,2021-12-23,MSFT,332.750000,336.390015,332.730011,334.690002,325.576385,333.200012,19617800,330.869000,330.786436,57.967254,330.869000,5.745903,342.360805,319.377195,0.155526,0.698788
8637,2022-10-19,TSLA,219.800003,222.929993,217.779999,222.039993,222.039993,220.190002,66571500,244.149001,240.152079,27.970226,244.149001,27.694001,299.537003,188.760998,-18.256350,-16.771065
5095,2024-06-28,META,517.150024,521.880005,503.839996,504.220001,502.886017,519.559998,15855100,498.963499,499.231704,57.078202,498.963499,12.902650,524.768799,473.158200,8.194616,7.722721
2328,2024-01-02,AMZN,151.539993,152.380005,148.389999,149.929993,149.929993,151.940002,47339400,149.968999,150.186364,60.813664,149.968999,3.354881,156.678762,143.259237,2.427522,2.823944
1930,2022-06-01,AMZN,122.255997,125.179001,120.622498,121.683998,121.683998,120.209503,127528000,111.743225,116.493696,68.412910,111.743225,5.938682,123.620588,99.865862,-5.343479,-8.253103


9254


## Loading into MySQL with historical data 

## Extracting Real time data
### Using Finnhub API to get real time data of the stock market , filling the Cassandra database every 10 seconds until 60seconds is reached for the past minute of data

In [33]:
from datetime import datetime
import finnhub

#using finhub api, use api key
finnhub_client = finnhub.Client(api_key="cvhdlvhr01qrtb3o0350cvhdlvhr01qrtb3o035g")

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA"]

def get_realtime_quotes():
    quotes = []
    for symbol in tickers:
        data = finnhub_client.quote(symbol)
        quotes.append({
            "Date": pd.to_datetime(data['t'], unit='s'),
            "Ticker": symbol,
            "Open": data['o'],
            "High": data['h'],
            "Low": data['l'],
            "Close": data['c'],           
            "Adj_Close": data['c'],       
            "Previous_Close": data['pc'],  
            "Volume": None               
        })
    return pd.DataFrame(quotes)


## Loading into Cassandra with Real time data

In [34]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['localhost'])
    session = cluster.connect()
    print("Cassandra connection established.")
except Exception as e:
    print("Connection error:", e)


# connecting to  the Cassandra session
cluster = Cluster(['localhost'])
session = cluster.connect()

# we are making a keyspace called "stock data" and inside the keyspace we have a table called real_time_quotes where we are pulling data realtime from the finnhub api
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS stock_data 
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")
session.set_keyspace('stock_data')
#schema for the cassandra csql table
session.execute("""
    CREATE TABLE IF NOT EXISTS real_time_quotes (
    ticker TEXT,
    date TIMESTAMP,
    open DOUBLE,
    high DOUBLE,
    low DOUBLE,
    close DOUBLE,
    adj_close DOUBLE,
    volume BIGINT,
    previous_close DOUBLE,
    PRIMARY KEY (ticker, date)
);
""")


Cassandra connection established.


In [35]:
#from cassandra.query import PreparedStatement
import time
#inserting the values from the api and clearing the white space and storing the columns as ticker, date, open, high, low, close, adj_close, and volume
def insert_to_cassandra(df):
    query = """
            INSERT INTO real_time_quotes (ticker, date, open, high, low, close, adj_close, volume, previous_close)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """

    for _, row in df.iterrows():
        session.execute(query, (
            row['Ticker'],
            row['Date'].to_pydatetime(),
            row['Open'],
            row['High'],
            row['Low'],
            row['Close'],
            row['Adj_Close'],
            row['Volume'] if pd.notna(row['Volume']) else 0,
            row['Previous_Close'] if pd.notna(row['Previous_Close']) else None
        ))  

#making a backup incase the cassandra server acts up
#def backup_to_csv(df, path='realtime_backup.csv'):
    #header = not os.path.exists(path)
    #df.to_csv(path, mode='a', index=False, header=header)



In [36]:
# pulls stocks values every 10 seconds, 14 times for about 2 min 33 seconds + some buffer time 
for _ in range(14):
    df_tenseconds = get_realtime_quotes()
    insert_to_cassandra(df_tenseconds)
    #backup_to_csv(df_hour)
    print(" For 10 seconds: Adding a new batch of 7 rows into Cassandra DB.")
    time.sleep(10)  # delay to avoid rate limiting with the API


InvalidRequest: Error from server: code=2200 [Invalid query] message="Undefined column name previous_close in table stock_data.real_time_quotes"

In [ ]:
#turning the cassandra table into a python pandas dataframe for datamanipulation
query = "SELECT * FROM stock_data.real_time_quotes"
rows = session.execute(query)
df_cassandra = pd.DataFrame(rows)
display(df_cassandra)

,ticker,date,adj_close,close,high,low,open,previous_close,volume
0,META,2025-03-28 20:00:00,576.7400,576.7400,601.7500,573.92,600.31,602.58,0
1,META,2025-04-07 19:23:59,517.0383,517.0383,539.3700,481.90,485.10,504.73,0
2,META,2025-04-07 19:24:12,516.4800,516.4800,539.3700,481.90,485.10,504.73,0
3,META,2025-04-07 19:24:21,516.4800,516.4800,539.3700,481.90,485.10,504.73,0
4,META,2025-04-07 19:24:29,516.4800,516.4800,539.3700,481.90,485.10,504.73,0
...,...,...,...,...,...,...,...,...,...
100,MSFT,2025-04-07 19:25:31,355.8300,355.8300,370.9999,344.79,350.88,359.84,0
101,MSFT,2025-04-07 19:25:43,355.8300,355.8300,370.9999,344.79,350.88,359.84,0
102,MSFT,2025-04-07 19:25:52,355.8300,355.8300,370.9999,344.79,350.88,359.84,0
103,MSFT,2025-04-07 19:26:01,355.9100,355.9100,370.9999,344.79,350.88,359.84,0


## Real Time Data: Data Cleaning 


In [ ]:
def cleaning(df):
    # making sure the date column is only in date time format
    df['date'] = pd.to_datetime(df['date'])
    #dropping rows that are missing ticker or date since its the most integral portion
    df = df.dropna(subset=['ticker', 'date'])
    # filling in the missing values using forward fill, or backfill when available 
    float_cols = ['open', 'high', 'low', 'close', 'adj_close', 'volume', 'previous_close']
    for col in float_cols:
        if col in df.columns:
            df[col] = df[col].ffill().bfill()
    # if there is still any remaining numerical columns (float)we can just average fill it 
    for col in float_cols:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].mean())
    # making sure theres no duplicate 
    df = df.drop_duplicates(subset=['ticker', 'date'], keep='last')
    # reseting index if we drop
    df = df.reset_index(drop=True)
    return df


#applying the cleaning function on our previous tile: df_cassandra dataframe
df_cassandracleaned = cleaning(df_cassandra)
display(df_cassandracleaned)
print("\nNull Count for Real Time Data (Extracted from Cassandra):\n")
print(df_cassandracleaned.isnull().sum())

,ticker,date,adj_close,close,high,low,open,previous_close,volume
0,META,2025-03-28 20:00:00,576.7400,576.7400,601.7500,573.92,600.31,602.58,0
1,META,2025-04-07 19:23:59,517.0383,517.0383,539.3700,481.90,485.10,504.73,0
2,META,2025-04-07 19:24:12,516.4800,516.4800,539.3700,481.90,485.10,504.73,0
3,META,2025-04-07 19:24:21,516.4800,516.4800,539.3700,481.90,485.10,504.73,0
4,META,2025-04-07 19:24:29,516.4800,516.4800,539.3700,481.90,485.10,504.73,0
...,...,...,...,...,...,...,...,...,...
100,MSFT,2025-04-07 19:25:31,355.8300,355.8300,370.9999,344.79,350.88,359.84,0
101,MSFT,2025-04-07 19:25:43,355.8300,355.8300,370.9999,344.79,350.88,359.84,0
102,MSFT,2025-04-07 19:25:52,355.8300,355.8300,370.9999,344.79,350.88,359.84,0
103,MSFT,2025-04-07 19:26:01,355.9100,355.9100,370.9999,344.79,350.88,359.84,0



Null Count for Real Time Data (Extracted from Cassandra):

ticker            0
date              0
adj_close         0
close             0
high              0
low               0
open              0
previous_close    0
volume            0
dtype: int64


### Note: Finnhub API does not provide real time volume in its free API model. Additionally accuracy of per second volume would be hard to predict. We would potentially need to drop this or exclude volume entirely from subsequent machine learning later

## Real Time Data: Feature Engineering
### Can only perform this on a weekday between 8am EST and 4pm EST. Otherwise we will not be able to get more than 1 iteration of results since stock market is closed realtime. 
1. Simple Moving Average (SMA)
2. Exponential Moving Average (EMA)
3. Relative Strength Index (RSI)
4. Bollinger Bands
5. MACD (Moving Average Convergence Divergence)
6. On-Balance Volume (OBV)
7. Volatility (ATR - Average True Range)


In [ ]:
def add_technical_indicators(df):
    df = df.sort_values(by=['ticker', 'date'])
    
    # calculating Simple Moving Average (SMA)
    df['SMA_5'] = df.groupby('ticker')['close'].transform(lambda x: x.rolling(window=5).mean())

    # calculating Exponential Moving Average (EMA)
    df['EMA_5'] = df.groupby('ticker')['close'].transform(lambda x: x.ewm(span=5, adjust=False).mean())

    # calculating Relative Strength Index (RSI)
    def compute_rsi(series, period=5):
        delta = series.diff()
        gain = delta.where(delta > 0, 0.0)
        loss = -delta.where(delta < 0, 0.0)
        avg_gain = gain.rolling(window=period).mean()
        avg_loss = loss.rolling(window=period).mean()
        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))
    
    df['RSI_5'] = df.groupby('ticker')['close'].transform(lambda x: compute_rsi(x))

    # calculating Bollinger Bands
    df['BB_Middle'] = df.groupby('ticker')['close'].transform(lambda x: x.rolling(window=5).mean())
    df['BB_Std'] = df.groupby('ticker')['close'].transform(lambda x: x.rolling(window=5).std())
    df['BB_Upper'] = df['BB_Middle'] + 2 * df['BB_Std']
    df['BB_Lower'] = df['BB_Middle'] - 2 * df['BB_Std']


    # calculating MACD
    ema12 = df.groupby('ticker')['close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
    ema26 = df.groupby('ticker')['close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
    df['MACD'] = ema12 - ema26
    df['MACD_Signal'] = df.groupby('ticker')['MACD'].transform(lambda x: x.ewm(span=9, adjust=False).mean())

    # calclulating On Balance Volume (OBV)
    def compute_obv(group):
        obv = [0]
        for i in range(1, len(group)):
            if group['close'].iloc[i] > group['close'].iloc[i - 1]:
                obv.append(obv[-1] + group['volume'].iloc[i])
            elif group['close'].iloc[i] < group['close'].iloc[i - 1]:
                obv.append(obv[-1] - group['volume'].iloc[i])
            else:
                obv.append(obv[-1])
        return pd.Series(obv, index=group.index)

    df['OBV'] = df.groupby('ticker')[['close', 'volume']].apply(compute_obv).reset_index(level=0, drop=True)

    # calculating Average True Range (ATR)
    df['prev_close'] = df.groupby('ticker')['close'].shift(1)
    df['high_low'] = df['high'] - df['low']
    df['high_prev_close'] = (df['high'] - df['prev_close']).abs()
    df['low_prev_close'] = (df['low'] - df['prev_close']).abs()
    df['TR'] = df[['high_low', 'high_prev_close', 'low_prev_close']].max(axis=1)
    df['ATR_5'] = df.groupby('ticker')['TR'].transform(lambda x: x.rolling(window=5).mean())

    # dropping the temporary columns we had 
    df.drop(columns=['prev_close', 'high_low', 'high_prev_close', 'low_prev_close', 'TR'], inplace=True)

    return df

df_cleaned_cass_features=add_technical_indicators(df_cassandracleaned)
display(df_cleaned_cass_features)


,ticker,date,adj_close,close,high,low,open,previous_close,volume,SMA_5,EMA_5,RSI_5,BB_Middle,BB_Std,BB_Upper,BB_Lower,MACD,MACD_Signal,OBV,ATR_5
30,AAPL,2025-03-28 20:00:00,217.9000,217.9000,223.81,217.68,221.67,223.85,0,NaN,217.900000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0,NaN
31,AAPL,2025-04-07 19:23:59,179.3600,179.3600,194.15,174.62,177.20,188.38,0,NaN,205.053333,NaN,NaN,NaN,NaN,NaN,-3.074416,-0.614883,0,NaN
32,AAPL,2025-04-07 19:24:08,179.2500,179.2500,194.15,174.62,177.20,188.38,0,NaN,196.452222,NaN,NaN,NaN,NaN,NaN,-5.456885,-1.583284,0,NaN
33,AAPL,2025-04-07 19:24:21,179.2500,179.2500,194.15,174.62,177.20,188.38,0,NaN,190.718148,NaN,NaN,NaN,NaN,NaN,-7.261306,-2.718888,0,NaN
34,AAPL,2025-04-07 19:24:29,179.2500,179.2500,194.15,174.62,177.20,188.38,0,187.00200,186.895432,0.0,187.00200,17.272573,221.547146,152.456854,-8.592275,-3.893565,0,21.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,TSLA,2025-04-07 19:25:31,229.7900,229.7900,252.00,214.25,223.78,239.43,0,229.96000,230.528524,0.0,229.96000,0.380132,230.720263,229.199737,-9.240855,-7.559781,0,37.75
26,TSLA,2025-04-07 19:25:44,229.7900,229.7900,252.00,214.25,223.78,239.43,0,229.79000,230.282350,0.0,229.79000,0.000000,229.790000,229.790000,-9.085016,-7.864828,0,37.75
27,TSLA,2025-04-07 19:25:52,229.7900,229.7900,252.00,214.25,223.78,239.43,0,229.79000,230.118233,NaN,229.79000,0.000000,229.790000,229.790000,-8.859388,-8.063740,0,37.75
28,TSLA,2025-04-07 19:26:01,229.9338,229.9338,252.00,214.25,223.78,239.43,0,229.81876,230.056755,100.0,229.81876,0.064309,229.947379,229.690141,-8.570180,-8.165028,0,37.75


## real time data: final cleaning 
Feature engineering introduced new features some of witch now contain nans such as SMA5, RSI5, Bollinger Bonds(BB upper, lower,std,middle), and ATR5. However OBV is 0 since it relies on volume which is not provided by 0. We would need to drop the volume and OBV column since it does not provide us data. However for: SMA_5, RSI_5, BB_Upper/Lower/Std/middle, ATR_5 they all have rollin windows of 5 values. So its suppose to have have NaN values in the first rows.

In [ ]:
def final_clean_realtime(df):
    # Drop OBV and volume since they're not useful
    df = df.drop(columns=['volume', 'OBV'], errors='ignore')

    # Drop rows where ALL of the engineered rolling features are NaN
    rolling_cols = ['SMA_5', 'EMA_5', 'RSI_5', 'BB_Middle', 'BB_Std', 'BB_Upper', 'BB_Lower', 'ATR_5']
    df = df.dropna(subset=rolling_cols, how='all')

    # Optional: reset index
    df = df.reset_index(drop=True)
    
    return df
df_cass_final = final_clean_realtime(df_cleaned_cass_features)
display(df_cass_final)
print('Which columns still have null?:')
print(df_cass_final.isnull().sum())


,ticker,date,adj_close,close,high,low,open,previous_close,SMA_5,EMA_5,RSI_5,BB_Middle,BB_Std,BB_Upper,BB_Lower,MACD,MACD_Signal,ATR_5
0,AAPL,2025-03-28 20:00:00,217.9000,217.9000,223.81,217.68,221.67,223.85,NaN,217.900000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN
1,AAPL,2025-04-07 19:23:59,179.3600,179.3600,194.15,174.62,177.20,188.38,NaN,205.053333,NaN,NaN,NaN,NaN,NaN,-3.074416,-0.614883,NaN
2,AAPL,2025-04-07 19:24:08,179.2500,179.2500,194.15,174.62,177.20,188.38,NaN,196.452222,NaN,NaN,NaN,NaN,NaN,-5.456885,-1.583284,NaN
3,AAPL,2025-04-07 19:24:21,179.2500,179.2500,194.15,174.62,177.20,188.38,NaN,190.718148,NaN,NaN,NaN,NaN,NaN,-7.261306,-2.718888,NaN
4,AAPL,2025-04-07 19:24:29,179.2500,179.2500,194.15,174.62,177.20,188.38,187.00200,186.895432,0.0,187.00200,17.272573,221.547146,152.456854,-8.592275,-3.893565,21.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,TSLA,2025-04-07 19:25:31,229.7900,229.7900,252.00,214.25,223.78,239.43,229.96000,230.528524,0.0,229.96000,0.380132,230.720263,229.199737,-9.240855,-7.559781,37.75
101,TSLA,2025-04-07 19:25:44,229.7900,229.7900,252.00,214.25,223.78,239.43,229.79000,230.282350,0.0,229.79000,0.000000,229.790000,229.790000,-9.085016,-7.864828,37.75
102,TSLA,2025-04-07 19:25:52,229.7900,229.7900,252.00,214.25,223.78,239.43,229.79000,230.118233,NaN,229.79000,0.000000,229.790000,229.790000,-8.859388,-8.063740,37.75
103,TSLA,2025-04-07 19:26:01,229.9338,229.9338,252.00,214.25,223.78,239.43,229.81876,230.056755,100.0,229.81876,0.064309,229.947379,229.690141,-8.570180,-8.165028,37.75


Which columns still have null?:
ticker             0
date               0
adj_close          0
close              0
high               0
low                0
open               0
previous_close     0
SMA_5             28
EMA_5              0
RSI_5             35
BB_Middle         28
BB_Std            28
BB_Upper          28
BB_Lower          28
MACD               0
MACD_Signal        0
ATR_5             28
dtype: int64


Analysis: We should keep NaNs during modeling prep later on since the remaining nans are only from rolling basis which is needed for our rolling features such as (SMA_5RSI_5,BB_Middle,BB_Std,BB_Upper,BB_Lower,ATR_5) to formulate a data point.

We should drop or fill NaNs only for visualization, or use .dropna() just before plotting specific columns that require to have full rows without nans

## Exploratory Data Analysis

## Machine Learning
### ARIMA for MySQL (historical) and LSTM for Cassandra (real time)